In [1]:
import pandas as pd
import numpy as np
import os
import glob
import json
from joblib import dump, load
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.ensemble import RandomForestRegressor

## Configuration

In [2]:
# Force cells in the robotic leg
CELLS = [3, 4, 7, 8]

# Path where the results are stored
RESULTS_PATH = '../../../../results'
# ID of the training and validation data resulting from this notebook, stored in RESULTS_PATH
DATA_ID = '0008_22072021'
# Hyperparameters search date
HS_DATE = '22072021'
# Number of folds in cross-validation
CV = 6

print('Model trained with data: ' + DATA_ID)

pd.set_option('display.max_columns', None)

Model trained with data: 0008_22072021


## Hyperparameters seach analysis

In [26]:
results_files_ls = glob.glob(os.path.join(RESULTS_PATH, DATA_ID, 'RF_{}'.format(HS_DATE), 'RF_{}_*.json'.format(HS_DATE)))

print('Number of results files: {}'.format(len(results_files_ls)))

Number of results files: 212


In [27]:
# Load all the results and generates a pandas dataframe
results_ls = []
for results_file in results_files_ls:
    with open(results_file) as json_file:
        results_dict = json.load(json_file)
        
    dict_aux = {}
    dict_aux['params_ID'] = results_dict['id']
    for key, value in results_dict['parameters'].items():
        dict_aux['param_' + key] = value
    for key, value in results_dict['cv_results'].items():
        dict_aux['__'.join([key, 'mean'])] = np.mean(value)
        dict_aux['__'.join([key, 'std'])] = np.std(value)

    results_ls.append(dict_aux)
        
results_df = pd.DataFrame(results_ls)
results_df

,params_ID,param_max_depth,param_max_features,param_min_samples_leaf,param_min_samples_split,param_n_estimators,fit_time__mean,fit_time__std,Train_Fx_MAE_mean__mean,Train_Fx_MAE_mean__std,Train_Fx_MAE_std__mean,Train_Fx_MAE_std__std,Train_Fx_MSE_mean__mean,Train_Fx_MSE_mean__std,Train_Fx_MSE_std__mean,Train_Fx_MSE_std__std,Train_Fx_R2_mean__mean,Train_Fx_R2_mean__std,Train_Fx_R2_std__mean,Train_Fx_R2_std__std,Train_Fy_MAE_mean__mean,Train_Fy_MAE_mean__std,Train_Fy_MAE_std__mean,Train_Fy_MAE_std__std,Train_Fy_MSE_mean__mean,Train_Fy_MSE_mean__std,Train_Fy_MSE_std__mean,Train_Fy_MSE_std__std,Train_Fy_R2_mean__mean,Train_Fy_R2_mean__std,Train_Fy_R2_std__mean,Train_Fy_R2_std__std,Valid_Fx_MAE_mean__mean,Valid_Fx_MAE_mean__std,Valid_Fx_MAE_std__mean,Valid_Fx_MAE_std__std,Valid_Fx_MSE_mean__mean,Valid_Fx_MSE_mean__std,Valid_Fx_MSE_std__mean,Valid_Fx_MSE_std__std,Valid_Fx_R2_mean__mean,Valid_Fx_R2_mean__std,Valid_Fx_R2_std__mean,Valid_Fx_R2_std__std,Valid_Fy_MAE_mean__mean,Valid_Fy_MAE_mean__std,Valid_Fy_MAE_std__mean,Valid_Fy_MAE_std__std,Valid_Fy_MSE_mean__mean,Valid_Fy_MSE_mean__std,Valid_Fy_MSE_std__mean,Valid_Fy_MSE_std__std,Valid_Fy_R2_mean__mean,Valid_Fy_R2_mean__std,Valid_Fy_R2_std__mean,Valid_Fy_R2_std__std
0,0053CDD6RU,50,0.5,10,20,10000,201.890455,1.777771,3.736151,0.112171,0.612247,0.038207,34.358118,2.026816,10.225985,0.764547,0.899108,0.006116,0.021815,0.002756,4.222927,0.032640,1.719379,0.038264,42.771542,0.599828,28.171035,0.701056,0.808222,0.003476,0.082249,0.004090,8.096202,0.727818,1.716026,0.549847,139.993334,26.463015,55.716064,24.496856,0.575438,0.087509,0.113608,0.045170,7.644204,0.415855,3.121978,0.442792,127.938003,10.030636,77.716880,12.841295,0.446647,0.071425,0.196926,0.100499
1,03LDFNOFGH,50,0.7,5,2,1000,30.840562,0.346915,2.650750,0.091164,0.425651,0.027319,17.590059,1.263665,5.101886,0.385094,0.948171,0.003863,0.011456,0.001543,3.047546,0.034458,1.178875,0.024499,22.656202,0.556882,14.389199,0.380046,0.893066,0.002750,0.050485,0.002207,7.969503,0.716748,1.670227,0.513883,136.023114,24.607592,53.232610,21.365071,0.584398,0.093226,0.110936,0.046993,7.439682,0.405848,3.040868,0.414780,122.112699,9.009124,74.259189,12.064453,0.467321,0.071489,0.195004,0.100667
2,061HB0EQT8,13,0.7,2,20,1000,28.579041,0.074339,3.701306,0.124426,0.581178,0.043149,30.530624,1.938824,8.382298,0.777786,0.909328,0.005910,0.021288,0.002607,4.218877,0.045854,1.710495,0.046744,40.590266,0.691653,27.352535,0.897998,0.816577,0.003311,0.080754,0.003808,8.096809,0.723109,1.670112,0.553595,138.758619,24.876122,53.505312,22.587005,0.576646,0.090707,0.111430,0.045420,7.613324,0.404981,3.109089,0.441891,126.621205,9.323795,77.020564,12.880402,0.449031,0.073805,0.196500,0.102723
3,0ECIQH6JIS,20,0.3,20,2,10000,112.413255,0.845373,4.870046,0.121530,0.828010,0.069738,57.782599,3.202461,18.961023,1.868171,0.833252,0.008480,0.031306,0.003976,5.466373,0.048516,2.308814,0.047691,70.862660,0.845228,47.607168,1.189424,0.701035,0.004139,0.116353,0.006726,8.244542,0.735303,1.721217,0.638640,145.031393,31.993636,58.421075,30.664391,0.565418,0.085620,0.110811,0.041208,7.919833,0.421240,3.256536,0.458714,136.494771,10.575385,83.757612,13.770335,0.420626,0.065226,0.192487,0.091239
4,0EXTVX4JY7,20,0.5,5,20,5000,105.714867,0.959715,3.447854,0.105891,0.548016,0.036185,28.880617,1.798413,8.154574,0.615347,0.914546,0.005362,0.019461,0.002397,3.922548,0.032420,1.570103,0.037402,36.733834,0.573335,24.005029,0.668698,0.831963,0.002941,0.075249,0.003455,8.060601,0.731816,1.685514,0.538854,138.252876,25.686860,54.051958,23.079408,0.578789,0.090948,0.112189,0.046226,7.586792,0.405176,3.100309,0.434901,126.218179,9.333729,76.868570,12.289142,0.453664,0.071122,0.193746,0.099320
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
207,BC1T31HIU7,20,0.2,2,20,5000,49.664191,0.247200,3.419503,0.105882,

In [28]:
# Sum up the scores by force axis in only one sortable score
for subset in ['Train', 'Valid']:
    for loss in ['MAE', 'MSE', 'R2']:
        results_df[subset + '_' + loss] = results_df[[subset + '_' + force + '_' + loss + '_mean__mean' for force in ['Fx', 'Fy']]].mean(axis=1)

In [29]:
# Sort the dataframe by the most relevant score
results_df = results_df.sort_values(['Valid_R2'], ascending=False)
results_df

,params_ID,param_max_depth,param_max_features,param_min_samples_leaf,param_min_samples_split,param_n_estimators,fit_time__mean,fit_time__std,Train_Fx_MAE_mean__mean,Train_Fx_MAE_mean__std,Train_Fx_MAE_std__mean,Train_Fx_MAE_std__std,Train_Fx_MSE_mean__mean,Train_Fx_MSE_mean__std,Train_Fx_MSE_std__mean,Train_Fx_MSE_std__std,Train_Fx_R2_mean__mean,Train_Fx_R2_mean__std,Train_Fx_R2_std__mean,Train_Fx_R2_std__std,Train_Fy_MAE_mean__mean,Train_Fy_MAE_mean__std,Train_Fy_MAE_std__mean,Train_Fy_MAE_std__std,Train_Fy_MSE_mean__mean,Train_Fy_MSE_mean__std,Train_Fy_MSE_std__mean,Train_Fy_MSE_std__std,Train_Fy_R2_mean__mean,Train_Fy_R2_mean__std,Train_Fy_R2_std__mean,Train_Fy_R2_std__std,Valid_Fx_MAE_mean__mean,Valid_Fx_MAE_mean__std,Valid_Fx_MAE_std__mean,Valid_Fx_MAE_std__std,Valid_Fx_MSE_mean__mean,Valid_Fx_MSE_mean__std,Valid_Fx_MSE_std__mean,Valid_Fx_MSE_std__std,Valid_Fx_R2_mean__mean,Valid_Fx_R2_mean__std,Valid_Fx_R2_std__mean,Valid_Fx_R2_std__std,Valid_Fy_MAE_mean__mean,Valid_Fy_MAE_mean__std,Valid_Fy_MAE_std__mean,Valid_Fy_MAE_std__std,Valid_Fy_MSE_mean__mean,Valid_Fy_MSE_mean__std,Valid_Fy_MSE_std__mean,Valid_Fy_MSE_std__std,Valid_Fy_R2_mean__mean,Valid_Fy_R2_mean__std,Valid_Fy_R2_std__mean,Valid_Fy_R2_std__std,Train_MAE,Train_MSE,Train_R2,Valid_MAE,Valid_MSE,Valid_R2
74,V24FH0UC1W,15,0.1,2,2,10000,55.433815,0.365775,2.296871,0.079446,0.396317,0.037432,11.732212,0.920337,3.385014,0.403451,0.965310,0.002982,0.008106,0.001118,2.815122,0.035594,1.208750,0.041252,18.799030,0.436795,13.565041,0.554038,0.920860,0.002329,0.034056,0.002084,7.651563,0.755338,1.584823,0.523207,124.222539,30.033897,49.943362,25.602384,0.622925,0.092683,0.096605,0.050295,7.252702,0.365828,3.034126,0.352281,116.585877,6.276876,73.496477,8.092109,0.519276,0.045046,0.153539,0.061081,2.555997,15.265621,0.943085,7.452133,120.404208,0.571101
98,7R02CFX48U,50,0.1,2,2,10000,59.208885,0.235305,1.850461,0.065129,0.328078,0.022181,8.687453,0.688850,2.672017,0.246840,0.974567,0.002000,0.005556,0.000717,2.228244,0.024905,0.926811,0.021581,12.490288,0.224083,8.491647,0.185703,0.945126,0.001149,0.024331,0.000832,7.650978,0.762719,1.585448,0.509426,124.582629,30.095099,50.004981,25.199315,0.621185,0.094385,0.097542,0.051336,7.228075,0.368219,3.020626,0.346609,116.276767,6.349649,73.131579,8.037413,0.519680,0.046109,0.153839,0.062024,2.039352,10.588870,0.959846,7.439526,120.429698,0.570432
191,Q23U62EVVF,50,0.1,2,2,700,4.252936,0.050852,1.856301,0.065399,0.330026,0.022740,8.780408,0.716602,2.722733,0.269942,0.974334,0.002039,0.005541,0.000725,2.233361,0.025622,0.928603,0.021984,12.584609,0.226383,8.532583,0.197831,0.944754,0.001143,0.024432,0.000867,7.658540,0.756697,1.581664,0.507181,124.765552,30.128455,49.912121,25.209502,0.620543,0.093914,0.097413,0.051405,7.229480,0.362742,3.022107,0.345232,116.251120,6.028564,73.064783,8.065914,0.519754,0.045449,0.154187,0.061513,2.044831,10.682509,0.959544,7.444010,120.508336,0.570149
64,TSHBOIBUXK,15,0.1,2,5,10000,56.254231,0.470965,2.404137,0.081543,0.411103,0.038023,12.927218,0.982097,3.705170,0.418853,0.961756,0.003161,0.008917,0.001218,2.930939,0.035743,1.246868,0.041493,20.315568,0.443831,14.475270,0.560693,0.913461,0.002337,0.037219,0.002066,7.662717,0.755368,1.586277,0.524433,124.501903,30.077141,50.081352,25.677183,0.622141,0.092595,0.096918,0.050186,7.268472,0.365178,3.039564,0.353230,117.035985,6.208482,73.789147,8.043481,0.517426,0.045094,0.153644,0.060964,2.667538,16.621393,0.937609,7.465595,120.768944,0.569784
176,6BZMRVEYPP,13,0.1,2,2,5000,26.528616,0.194066,2.897698,0.089359,0.483617,0.053530,17.259588,1.185368,4.746406,0.658424,0.948507,0.004127,0.012973,0.001703,3.508937,0.038752,1.529810,0.056306,28.292472,0.527744,21.109705,1.003738,0.883873,0.003317,0.051255,0.003657,7.674408,0.747361,1.579029,0.548122,124.114587,29.909213,49.686382,26.139441,0.623753,0.090377,0.095367,0.048659,7.317514,0.365817,3.063096,0.361134,117.733043,6.042328,74.332495,8.317944,0.515317,0.043497,0.153876,0.059840,3.203317,22.776030,0.91619

In [30]:
best_params = dict(results_df.iloc[0][[col for col in results_df.columns if 'param_' in col]])
best_params = {key.replace('param_', ''): value for key, value in best_params.items()}
print('Best parameters: {}'.format(best_params))

Best parameters: {'max_depth': 15, 'max_features': 0.1, 'min_samples_leaf': 2, 'min_samples_split': 2, 'n_estimators': 10000}


## Best model

In [17]:
# Load data
X_train = np.load(os.path.join(RESULTS_PATH, DATA_ID, 'data', 'X_train_{}.npy'.format(DATA_ID)))
X_test = np.load(os.path.join(RESULTS_PATH, DATA_ID, 'data', 'X_test_{}.npy'.format(DATA_ID)))
Y_train = np.load(os.path.join(RESULTS_PATH, DATA_ID, 'data', 'Y_train_{}.npy'.format(DATA_ID)))
Y_test = np.load(os.path.join(RESULTS_PATH, DATA_ID, 'data', 'Y_test_{}.npy'.format(DATA_ID)))

In [18]:
# Setup the model with the best parameters
model = RandomForestRegressor(**best_params, random_state=0, n_jobs=-1, verbose=1)

model.fit(X_train, Y_train)

# Save the model
dump(model, os.path.join(RESULTS_PATH, DATA_ID, 'RF_{}'.format(HS_DATE), 'RF_best_model_{}_{}.joblib'.format(HS_DATE, DATA_ID))) 

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    0.6s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:    2.2s
[Parallel(n_jobs=-1)]: Done 434 tasks      | elapsed:    5.2s
[Parallel(n_jobs=-1)]: Done 784 tasks      | elapsed:    9.3s
[Parallel(n_jobs=-1)]: Done 1234 tasks      | elapsed:   14.3s
[Parallel(n_jobs=-1)]: Done 1784 tasks      | elapsed:   20.5s
[Parallel(n_jobs=-1)]: Done 2434 tasks      | elapsed:   28.2s
[Parallel(n_jobs=-1)]: Done 3184 tasks      | elapsed:   37.2s
[Parallel(n_jobs=-1)]: Done 4034 tasks      | elapsed:   47.5s
[Parallel(n_jobs=-1)]: Done 4984 tasks      | elapsed:   59.5s
[Parallel(n_jobs=-1)]: Done 6034 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done 7184 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done 8434 tasks      | elapsed:  1.7min
[Parallel(n_jobs=-1)]: Done 9784 tasks      | elapsed:  2.0min
[Parallel(n_jobs=-1)]: Done 10000 out of 

['../../../../results/0007_19072021/RF_21072021/RF_best_model_21072021_0007_19072021.joblib']

In [20]:
train_preds = model.predict(X_train)
test_preds = model.predict(X_test)

results = {
    'Train': {
        'MAE': mean_absolute_error(Y_train, train_preds, multioutput='raw_values'),
        'MSE': mean_squared_error(Y_train, train_preds, multioutput='raw_values'),
        'R2': r2_score(Y_train, train_preds, multioutput='raw_values')
    },
    'Test': {
        'MAE': mean_absolute_error(Y_test, test_preds, multioutput='raw_values'),
        'MSE': mean_squared_error(Y_test, test_preds, multioutput='raw_values'),
        'R2': r2_score(Y_test, test_preds, multioutput='raw_values')
    }       
    
}

# # Display the score for each axis of each force cell
# for subset in ['Train', 'Test']:
#     for f, force in enumerate(['Fx', 'Fy', 'Fz']):
#         for c in range(N_CELLS):
#             for loss in ['MAE', 'MSE', 'R2']:
#                 scores = [results[subset][loss][i + f] for i in range(0, N_CELLS * 3, 3)]
#                 print('{} {}{}{} {}: {:.4f}'.format(subset, force[0], c + 1, force[-1], loss, scores[c]))
# print('\n')

# Display the score mean and standard deviation of each axis
for subset in ['Train', 'Test']:
    for f, force in enumerate(['Fx', 'Fy']):
        for loss in ['MAE', 'MSE', 'R2']:
            scores = [results[subset][loss][i + f] for i in range(0, len(CELLS) * 2, 2)]
            print(' '.join([subset, force, loss]) + ': {:.4f} ± {:.4f}'.format(np.mean(scores), np.std(scores)))

[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:    0.2s
[Parallel(n_jobs=8)]: Done 434 tasks      | elapsed:    0.4s
[Parallel(n_jobs=8)]: Done 784 tasks      | elapsed:    0.6s
[Parallel(n_jobs=8)]: Done 1234 tasks      | elapsed:    0.9s
[Parallel(n_jobs=8)]: Done 1784 tasks      | elapsed:    1.4s
[Parallel(n_jobs=8)]: Done 2434 tasks      | elapsed:    1.9s
[Parallel(n_jobs=8)]: Done 3184 tasks      | elapsed:    2.5s
[Parallel(n_jobs=8)]: Done 4034 tasks      | elapsed:    3.1s
[Parallel(n_jobs=8)]: Done 4984 tasks      | elapsed:    3.8s
[Parallel(n_jobs=8)]: Done 6034 tasks      | elapsed:    4.6s
[Parallel(n_jobs=8)]: Done 7184 tasks      | elapsed:    5.4s
[Parallel(n_jobs=8)]: Done 8434 tasks      | elapsed:    6.3s
[Parallel(n_jobs=8)]: Done 9784 tasks      | elapsed:    7.2s
[Parallel(n_jobs=8)]: Done 10000 out of 10000 | elapsed:

Train Fx MAE: 2.2904 ± 0.4356
Train Fx MSE: 13.1327 ± 3.9882
Train Fx R2: 0.9611 ± 0.0104
Train Fy MAE: 2.7660 ± 1.1741
Train Fy MSE: 19.6227 ± 14.1450
Train Fy R2: 0.9152 ± 0.0379
Test Fx MAE: 7.9542 ± 2.5406
Test Fx MSE: 164.2414 ± 107.3285
Test Fx R2: 0.6927 ± 0.0674
Test Fy MAE: 7.0967 ± 2.7796
Test Fy MSE: 117.8615 ± 70.1520
Test Fy R2: 0.5324 ± 0.1616


[Parallel(n_jobs=8)]: Done 9784 tasks      | elapsed:    2.2s
[Parallel(n_jobs=8)]: Done 10000 out of 10000 | elapsed:    2.3s finished


In [ ]:
# plt.figure(figsize=(20,15))
# plt.scatter(Y_train[:, 3], Y_train[:, 4], label='true', alpha=0.3)
# plt.scatter(train_preds[:, 3], train_preds[:, 4], label='preds', alpha=0.3)
# plt.legend()
# plt.show()

# plt.figure(figsize=(20,15))
# plt.scatter(Y_train[:100, 3], Y_train[:100, 4], label='true', alpha=0.3)
# plt.scatter(train_preds[:100, 3], train_preds[:100, 4], label='preds', alpha=0.3)
# plt.legend()
# plt.show()

# plt.figure(figsize=(20,15))
# plt.scatter(Y_test[:, 3], Y_test[:, 4], label='true', alpha=0.3)
# plt.scatter(test_preds[:, 3], test_preds[:, 4], label='preds', alpha=0.3)
# plt.legend()
# plt.show()